Install

In [1]:
%pip install TTS -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re

def split_and_recombine_text(text, desired_length=200, max_length=300):
    """Split text it into chunks of a desired length trying to keep sentences intact."""
    # normalize text, remove redundant whitespace and convert non-ascii quotes to ascii
    text = re.sub(r'\n\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[“”]', '"', text)

    rv = []
    in_quote = False
    current = ""
    split_pos = []
    pos = -1
    end_pos = len(text) - 1

    def seek(delta):
        nonlocal pos, in_quote, current
        is_neg = delta < 0
        for _ in range(abs(delta)):
            if is_neg:
                pos -= 1
                current = current[:-1]
            else:
                pos += 1
                current += text[pos]
            if text[pos] == '"':
                in_quote = not in_quote
        return text[pos]

    def peek(delta):
        p = pos + delta
        return text[p] if p < end_pos and p >= 0 else ""

    def commit():
        nonlocal rv, current, split_pos
        rv.append(current)
        current = ""
        split_pos = []

    while pos < end_pos:
        c = seek(1)
        # do we need to force a split?
        if len(current) >= max_length:
            if len(split_pos) > 0 and len(current) > (desired_length / 2):
                # we have at least one sentence and we are over half the desired length, seek back to the last split
                d = pos - split_pos[-1]
                seek(-d)
            else:
                # no full sentences, seek back until we are not in the middle of a word and split there
                while c not in '!?.\n ' and pos > 0 and len(current) > desired_length:
                    c = seek(-1)
            commit()
        # check for sentence boundaries
        elif not in_quote and (c in '!?\n' or (c == '.' and peek(1) in '\n ')):
            # seek forward if we have consecutive boundary markers but still within the max length
            while pos < len(text) - 1 and len(current) < max_length and peek(1) in '!?.':
                c = seek(1)
            split_pos.append(pos)
            if len(current) >= desired_length:
                commit()
        # treat end of quote as a boundary if its followed by a space or newline
        elif in_quote and peek(1) == '"' and peek(2) in '\n ':
            seek(2)
            split_pos.append(pos)
    rv.append(current)

    # clean up, remove lines with only whitespace or punctuation
    rv = [s.strip() for s in rv]
    rv = [s for s in rv if len(s) > 0 and not re.match(r'^[\s\.,;:!?]*$', s)]

    return rv

def add_space_after_dot(text):
    # Use regular expression to find dots followed by non-space characters
    # and replace them with dot followed by space
    modified_text = re.sub(r'\.(?!\s|$)', '. ', text)
    # Replace multiple spaces with a single space
    modified_text = re.sub(r'\s+', ' ', modified_text)
    return modified_text

def preproces_abbreviations(text):
    return re.sub(r'\b([A-Z]+)\b', lambda match: '-'.join(match.group(1)), text)
    
    
eng_text = """Year nineteen seventyfive The anti-Brezhnev stagnation. The Soviet Union signs the Helsinki Conclusion Act, according to which, in particular, human rights and freedoms must be upheld. At the same time, the USSR undergoes the development of punitive psychiatry. Thousands of absolutely healthy aliens are subjected to forced treatment in prison-type special institutions. One of these prisons is Ukrainian mathematician, publicist, and literary scholar Leonid Plush, who believed in the possibility of building socialism with a human face.He later summed up his tragic experience of being in the walls of the terrible Dnipropetrovsk Special Psychiatric Hospital as follows. "Recognizing yourself as a patient is the first condition for recovery. Then, you must repent of your harmful activities. However, it is not the doctors who release you, but the court. The court may approve that the patient needs further treatment. In essence, the diagnosis is made by the KGB. The treatment is assigned to the KGB. My wife was told by the KGB that if she would behave more calmly, she would be given fewer doses. And the KGB cured her." 

Why was it sometimes worse to end up in such hospitals than in a prison cell or a labor colony of a severe regime? What measures of influence were applied to people, and how large-scale was this repressive practice? Stay tuned. Glory to Ukraine, My name is Vladlen Maraev, and you are watching the story without myths, where we tell you about the past of Ukraine and the world without embellishments and falsifications. Subscribe to our channel and click the bell so as not to miss new videos.

May the Ukrainian YouTube bloom! The topic of the issue was chosen by our patrons. 46% supported the option of penal psychiatry in the USSR. Thank you to everyone who took part in the voting.

Penal psychiatry. Cruel torture in the guise of good intentions in order to cure the patient of harmful ideas. In the Soviet Union, it was believed that if you do not like the apparent political and social aura, you are clearly not all right. 
After all, you live in the best and freest country. For the first time in world history, state workers are happy. Nikita Khrushchev is credited with the famous phrase that only God-willing people can not like socialism.

However, the use of psychiatry for political purposes was not an invention of the USSR. It was practiced much earlier in the Russian Empire. In 1799, the metropolitan of Kyiv Yerofei declared God-willing the outstanding composer Artemiy Vedel, in the notes of which, God-willing, they found a prophecy about the assassination of the Emperor Paul I. Vedel was taken into custody and, without a formal acquittal, he was kept in isolation as a spiritual enemy almost until his death.
Interestingly, in 1801, Paul was indeed killed during a palace coup in St. Petersburg. Vedel passed away in 1808 and was buried in the Shchekovitsky Cemetery, later destroyed by the Bolsheviks, for the construction of a radio tower. Neither the grave, nor any living portrait of the composer were preserved.In 1836, a philosopher and publicist Petr Cheadaev was detained in Moscow. He published one of his philosophical letters in the magazine Telescope. In this work, Cheadaev criticized Russia as a country of gloomy existence, where there is no internal development, unlike European civilization, the basis of which is the ideas of justice, law, duty, and order. Emperor Nikolai Palkin was enraged. He called Cheadaev's letter a dream of God-willing, after which the Telescope was closed, the editor-in-chief was sent to Ust-Sysolsk, today's Syktyvkar, and the careless censor, who had let such a crumpled letter to be printed, was fired. Cheadaev himself was executed in accordance with the words of the tsar. For a year, he was put under house arrest as a spiritual enemy under the supervision of doctors. In response, Cheadaev wrote an apology of God-willing, but he could not publish it for life. But not only Russia. In 1851, American doctor Samuel Cartwright, who worked in the southern slave-owning states of Louisiana and Mississippi, proposed to use new psychiatric diagnoses. Dysesthesia of Ethiopians, i.e. pathological laziness among slaves, and drapetomania, a slave train to escape from plantations. Cartwright believed that dark-skinned slaves should be treated as if they were great children. But if they show an intention to escape from such a paradise, they should be given preventive treatment, such as beating with a baton. If you treat them kindly, feed them well and dress them, give them enough firewood and coal to keep them warm all night, give each family their own land, do not allow them to run to neighbors at night and drink alcoholic and narcotic substances, they are very easy to be controlled. Easier than any other people in the world.

But if any of them at any time shows an intention to become on the same level with their master or supervisor, humanity and their own good demand that they be punished until they fall into the state of obedience intended for them. They should only be kept in such a state and treated as if they were children, to prevent escape and cure from it. In 1937, a rather resonant case took place in Suriname, which was then a Dutch colony.

Pro-communist activist Louis Dodel, the author of anti-colonial pamphlets, wanted to negotiate with the governor on the establishment of poor and unemployed workers, but he was not allowed into the residence. The next day, he came smeared with white clay, as it was said that only whites were allowed there. The governor ordered Dodel to be sent to the psychiatric hospital Wolfenbüttel in Paramaribo, where, in particular, electrocardiotherapy was used for the unfortunate.

Dodel stayed in the hospital for more than 42 years and was released only a few days before his death. In the 20th century, criminal psychiatry began to be actively used in the People's Republic of China, in Cuba under Fidel Castro's regime, as well as in communist Romania, where people accused of spreading fascism, Zionism and religious propaganda were tried to be re-educated. The Bolshevik regime used methods of criminal psychiatry practically from the very beginning of its existence.

At first, these were isolated cases. For example, the story of the terrorist Maria Spiridonova, who in 1906 shot the black-haired adviser of the Tambov governor, and in 1918 supported the uprising of leftists against the Bolsheviks. She was arrested by the Czechs and, on the orders of Felix Dzerzhinsky, put in the Prechistensky Psychiatric Hospital, where she was diagnosed with hysterical psychosis.

In fact, she was released a few months later, but then she was repeatedly imprisoned and, finally, executed in the Medvedivsky Forest near Orlo in September 1941. Dear viewers, as you know from the news, this year we will have the first of our Western allies who will defend the sky over Ukraine. We all understand that they will become the main targets of the enemy.

Our task is to reliably protect the airfields from which they will soon take off. To do this, together with colleagues from the volunteer group 35, we are organizing a collection of encrypted, barely visible radio stations, which will allow us to form the first and most important stage of defense of the fighters. Follow the QR code on the screen or the link in the video description.

The goal is 3,600,000 hryvnias. This is the total cost for all radio stations. Let's jointly help the air forces of the Armed Forces of Ukraine.

In the wake of Stalin's repressions, some even managed to simulate mental illness in order to at least save their lives. Thus, in 1934, Volodymyr Sosyura ended up in a psychiatric hospital in Kharkiv, known as Saburova Dacha. Ukrainian writer and party activist Ivan Kulik wrote Sosyura is mentally ill. 

He has schizophrenia, a manic state, elements of erotic psychosis. He is gambling, compromising Soviet institutions. In conclusion, as you know, Sosyura survived the era of terror, although he was forced to go through numerous humiliation and compromises with conscience.

To write pathos poems about Stalin, Voroshilov and the GPU, the Kremlin tyrant's poster in one of the letters read, Father, save me! Instead, Kulik, who had a successful career on the party line, was executed in 1937 as an English spy. Sosyura even in the first post-war years went to the Pavliv Psychiatric Hospital, which is in Kyiv. The future film director Alexander Muratov, who visited him there with his father, testified that it was a simulation.

What can I say? Volodymyr Mykolaivych did not make me feel like a mentally ill person. I saw people who were out of their minds. And before that, and after that.

It was a completely different picture. What Sosyura said then could only be said by a completely healthy and very intelligent person. But like most genius poets, he did not behave like ordinary people.

In the 1930s, the patient of several psychiatric hospitals in Kyiv and Moscow was the talented Ukrainian writer Todos Osmachko, the author of numerous poems and the story The Elder Boyar, Rotunda of the Soul-Lovers. It helped him to avoid an imminent execution. During the Second World War, Osmachko found himself under Nazi occupation and managed to leave for Germany.

And after the war in the USA, away from the Stalinist regime. However, among his biographies there is still no, and probably never will be, unanimity. Was Osmachko a simulator? Did he really suffer from the disease? Or did he get it as a result of forced treatment? It is known that even overseas he was so afraid of the persecution of NKVD members that he lived with a constant fear of being poisoned.

In the premises, he had a hearing aid and some machines that could read human thoughts. A close friend of him, a doctor and a writer Maria Kayvan, recalled, Osmachko always claimed that his stay in a psychiatric hospital was conscious of the simulation of the disease, and some moments in his life indicated that he had symptoms of schizophrenia even before. As a doctor, he had a lot of trust in me, so he often went to the hospital with his mostly imaginary diseases.

He was a hypochondriac, who suspected himself of various severe diseases. In fact, he had a healthy and surprisingly durable body. For many years of his life, prison was strongly marked on his psyche, but not on his physical health.

One of the most original writers of the 1920s and 1930s, Daniel Harms, a master of absurdity and black humor, was arrested in August 1941 in Leningrad. The reason was a denunciation, according to which Harms allegedly stated that the USSR would lose the war with Germany immediately, that he himself was not going to serve in the Red Army under any circumstances, and that if he was forcibly mobilized, he would shoot his commanders from a machine gun. It is not known whether Harms really said this, but the military tribunal decided to isolate him in the psychiatric ward of the notorious Leningrad prison of Khresty.

Perhaps, like Sosiuri, he managed to convincingly simulate the devil. If so, then Harms made a fatal miscalculation. On February 2, 1942, he died in this ward.

At that time, the USSR already had a special prison-type psychiatric hospital in Kazan. In 1939, it was transferred by Beria to the NKVD. In particular, from 1943, the president of the occupied Soviet Estonia, Konstantin Piatsa, was received there, about whom we told in the video After the war, he was transferred to a similar facility in the village of Burashevo near Kalininy, Tver.

Piatsa continued to call himself the president, but the regular medical personnel did not even take his words seriously and thought it was a nickname for a prisoner. He never went to freedom. He died in 1956, a month before the official declaration of de-Stalinization at the XX Congress of the CPSU.

Dear friends, the team of Stories without Myths sends you a sincere thank you for your support. And let me remind you that you can help create new videos for you through the Patreon platform, BuyMeACoffee, or through sponsorship on YouTube. By becoming a patron or sponsor of our channel, you will receive early access to new videos, to the lists of recommended sources and literature, and will be able to vote on the topic for the next month.

You can help Stories without Myths by transferring money to PrivatBank, Monobank, or PayPal. Requisites and links are in the description of this video. During Stalin's lifetime, special mental hospitals or separate prisons were also created in Moscow, Butyrka, Tomsk, Sychovka in Smolensk region, Shatsk in Ryazan region, Chistopol in the Tatar SSR.

A total of thousands of people were received there. They were given insulin-comatose therapy, injections of rosin-cleansed sulfuric acid, wet towels, and other methods of physical influence. As a result, many prisoners quickly turned into seriously ill people, crippled, or simply died.

So, Admiral Lvov-Haller was imprisoned for the fact that during the war he allegedly handed over to his allies, the Americans and the British, some materials on torpedo weapons and maps of the two islands and the southern coast of Kamchatka. In just less than four months of his stay in the Kazan hospital, he died. The cause of his death was documented in the documents as tuberculosis, pleurisy, and deep organic delirium.

But the most massive use of the SSR's punitive psychiatry came in the 1960s. Special prisons of the prison type began to be built throughout the Union, or the already existing institutions were re-profiled, or separate cells were created in the hospitals of the Ministry of Health, which were controlled by the Ministry of Internal Affairs and the KGB. The decision was approved by the closed decisions of the Central Committee of the CPSU and the Council of Ministers of the USSR.

The main initiator was the head of the KGB and the future General Secretary Yuriy Andropov, motivating this need to protect the interests of the Soviet state and the social order. Special mental hospitals became one of the main and at the same time the most advanced instruments of the state in the fight against dissidents and any dissidents. If the place of deprivation of freedom could be returned after the termination of the prison, then the time spent in the clinic could not be regulated by anything.

At the same time, such a method allowed the Soviet authorities to declare to the world community that there were no political prisoners in the USSR, and also to discredit the opposition in the eyes of their society. They said they were not ideological fighters and not martyrs who suffer from repressions, but mentally ill people. Many prisons were diagnosed with slow-moving schizophrenia, introduced by the academician of the Academy of Medical Sciences of the USSR Andriy Snezhnevsky.

This concept did not gain recognition in the world of psychiatry, but opened a wide range of manipulations and abuse. Psychiatrist Khristina Shavak, a lecturer at the Ukrainian Catholic University, wrote in her book No One is in the Forest. Slow-moving schizophrenia is a disease that seems to exist, but seems not to exist.

It is devoid of bright symptoms of ordinary schizophrenia, which is used to diagnose it. Instead, it is endowed with a wide range of manifestations and variants, which are described in the criteria of other mental disorders and in the understanding of the relative mental norm. In fact, such a diagnosis can be given to anyone at will.

Specialists in psychiatry could be found under such articles of the Criminal Code as anti-Soviet agitation and propaganda, dissemination of allegedly false guesses that discredit the Soviet state and social authorities, organization or active participation in group actions that violate public order, violation of the state coat of arms or flag. Lawyers could not file a complaint, as they were given a medical conclusion that their client suffers from schizophrenia or paranoia. One of the most famous victims of criminal psychiatry was Major General Petro Hryhorenko.

Like many other human rights defenders, he did not speak up against the Soviet authorities as such. He only insisted on the adherence to Lenin's principles and norms, democratization of elections, disclosure of historical truth, in particular, on the real causes of defeats and catastrophic losses of the Red Army in World War II. He defended the right of the deported Crimeans to return to their historical homeland.

In conclusion, he was deprived of awards, punished in a row, twice directed to forced treatment, first in Leningrad, then in Cherniakhovsk, Kaliningrad region. An expert commission concluded that the ex-general suffers from mental illness in the form of a pathological, paranoid development of personality with the presence of ideas of reformation. That's right, only a mentally ill person can desire reform in the USSR.

Hryhorenko recalled his experience of being in hospitals. For a mentally healthy person, psychiatry is especially scary because it is placed in the environment of people with a deformed psyche. But not less scary is complete ineptitude and hopelessness.

The patient does not even have those miserable rights that the prisoners have. He has no rights at all. Doctors can do anything with him, no one will get poisoned, no one will protect him.

None of his complaints or the complaints of those who stay with him will leave the hospital. After his release, Hryhorenko did not quit his law enforcement activities. He became a co-founder of the Moscow and Ukrainian Helsinki Group.

In 1977, he was allowed to go to the United States for treatment, after which he was deprived of Soviet citizenship. Psychiatric examination in the United States did not find a sign of mental illness in him. Already mentioned at the beginning of the story Leonid Plush, for two and a half years he was isolated in the Dnipropetrovsk Special Psychiatric Hospital with a diagnosis of slow-moving schizophrenia.

Doctor Elva Kamenetska, who was nicknamed Elza Koch or Elva Ludaidka by the prisoners, openly declared to him, nothing, soon we will cure you from political delirium. The stay in the psychiatric hospital is dedicated to a separate section of Plush's memoirs in the Carnival of History. How does Haloperidol work, I see in the cellmates in quarantine.

One is all wrinkled from the convulsions. He can't lie down, fainted. The head turned sideways, the eyes are swollen.

The second is suffocating, he pulled out his tongue. The third shouts, calls for a nurse, asks for a corrector, medicines that remove the physical consequences of Haloperidol. It turns out that Haloperidol is given so much to scare, break the will to resist and reveal the simulants.

Punishment. Tied to the bed with belts, from a few hours to the day. Increase the dose of neuroleptics.

The nurses beat. But the worst punishment was considered a syringe. After a syringe injection, the person's temperature rose to 40 degrees.

The place of the injection hurts, you can neither walk nor lie down. Every time the dose of the syringe is increased. Then they gradually begin to decrease.

In the 12th department, of course, they gave a course of 10-15 injections. Everyone was talking about the 9th department with fear. There is a course of up to 20-25 injections and even more doses.

Viktor Borovsky, a student of the Slavyansky Pedagogical Institute in Donetsk Oblast, In 1975, he gave a speech about the 20th Congress of the CPSU and the policy of de-Stalinization. But those were the times of Brezhnev, when the authorities were actively rehabilitating the image of the Kremlin figure in public consciousness. So the student soon ended up in a psychiatric hospital in Slavyansk.

Here are the terrible feelings that came from just one syringe injection. I almost did not feel the injection itself. But when I saw it, it immediately threw me into a cold sweat.

Lights went out in my eyes. I felt a slight drunkenness. My legs were like cotton wool.

It was easy to walk. The dream was hot. All night before me there were some terrible pictures.

The pain was no longer in one place, but rolled into my leg and neck. The sun was already setting. The bed was blocked by diseases.

But I could not bring my head down, as if I wanted to sleep, and my bones were breaking. Something was ringing in my head. My lips dried up and wrinkled.

Nail holes and nails themselves became white. My face and ears were very swollen. I closed my eyes.

But a nurse came up and pulled out a thermometer. Setting the thermometer, I seemed to fall asleep again. The nurse came in.

I took out the thermometer and tried to look around. But in the eyes of the sea of ​​the body I only saw something over 40. Valeriya Novodvorskaya was forcibly treated in the Kazan Special Psychiatric Hospital with a diagnosis of slow-moving schizophrenia and paranoid personality disorder for more than a year and a half.

She was spreading anti-Soviet leaflets herself. According to her father, as a result of such treatment, she forever lost the ability to give birth to children. The story of Mykhailo Yakubivskyi is absolutely terrible.

He ended up in a psychiatric hospital in Hlavaz near Kyiv at the age of 15. Due to the fact that he reacted negatively to the invasion of the USSR into Czechoslovakia and because of the bullying of his classmates, he stopped going to school. They probably injected me with neuroleptics, haloperidol.

Well, they definitely injected me with aminazine. At night, they brought me to such a state that I had a temperature of 42 degrees. Angina.

After that, the room of wet nests on the fourth floor. Terrible, where there were children. Terrible picture.

I saw epileptics. There are children masturbating. They wear pants.

It all stinks. I was treated there. Then insulin.

When insulin is taken, the temperature rises. It can cause nausea. Finally, Mykhailo was discharged.

He graduated from school and was able to enter the philological faculty, where he was considered to be sympathetic to dissidents, and twice got forced treatment. After the restoration of Ukraine's independence in 1993, the expert commission of the Association of Psychiatrists of Ukraine recognized Yakubivskyi as mentally healthy. The victims of repressive psychiatry of the late USSR were many well-known Ukrainian dissidents.

In addition to those already mentioned today, these are Zenovy Krasivskyi, Anatoliy Lupynis, Taras Melnychuk, Hanna Mykhailenko, Mykola Plokhotnyuk, Viktor Rafalsky, Vasyl Ruban, Iosyp Terelya, and others. In general, Vakhtang Kipiani told in detail about the struggle of Ukrainian dissidents on our channel in a video, which I recommend you to watch at the link in the upper right corner of the screen. Dear viewers, The Story Without Myths has its own merch.

These are warm, comfortable and high-quality hoodies with the slogan of our channel High-quality Ukrainian YouTube. To order, go to our Instagram direct or to the Telegram channel. The link is in the video description.

In 1988, 16 special-purpose psychiatric hospitals were transferred to the Ministry of Health of the USSR only as part of the reconstruction policy. Five of them were liquidated. According to various sources, from 776 thousand to 2 million patients were removed from the psychiatric facility.

It was one of the basic requirements of the adoption of Soviet psychiatrists to the World Psychiatric Association. In 1989, an article was removed from the Criminal Code stating that anti-Soviet agitation and propaganda and incitement to the Soviet power were considered as a socially dangerous activity. It is impossible to determine the exact number of patients from penitentiary psychiatry.

According to the Dutch historian and human rights activist Robert Van Voren, this is about a third of all political prisoners. According to researcher Anatoliy Prokopenko, there were from 15 to 20 thousand political prisoners. But hundreds of thousands became victims of abuse of psychiatry, even though they were not foreigners.

The consequences of this are still felt in our society. Hrystyna Shalak notes. The trauma from penitentiary psychiatry of the Soviet Union left a mark in linguistics, in phrases similar to prison jargonisms.

People are afraid that they will be put in the guise of the police, stabbed not only with knives, but also with injections, locked up like in prison, turned into a vegetable, deprived not only of will, but also of personality, of their own self. This trauma caused fear of the medical system as a whole. Doctors are perceived as a direct, unpredictable figure you depend on.

A psychiatrist of collective imagination, somehow in the shape of a policeman dressed in a white robe, but more of an employee of the KGB. The drugs he prescribes are equated to poison. His diagnosis is a court ruling.

The modern Russian Federation continues the traditions of the totalitarian Soviet regime. International human rights organizations, mass media, independent psychiatric associations have repeatedly denoted the facts of forced psychiatric treatment of opposition members, protesters, human rights activists. For example, the critics of Putin's policy in Chechnya gave the stationary treatment to the journalist Andriy Novikov, for his participation in the rallies in Bolotnaya Square, Mikhail Kosenko and Maksym Panfilov, for the protest against Putin's actions, Yakutsk shaman Alexander Gabyshev, for his participation in the Crimean-Tatar National Rus' on the temporarily occupied peninsula, Ilmiy Umerov and Nariman Dzhelyau were arrested for repressive psychiatry.

Already after the beginning of a large-scale invasion of Ukraine for anti-war propaganda, the Yekaterinburg artist Leonid Chornyh and the resident of St. Petersburg Victoria Petrova was forced to undergo psychiatric treatment And this is just a small selection of examples. Dear viewers, in order to make Ukrainian YouTube flourish, please subscribe to the story without myths, click the bell and like this video. Thank you for watching. 

Glory to Ukraine!"""

tts_text = split_and_recombine_text(eng_text)
tts_text = [preproces_abbreviations(add_space_after_dot(text)) for text in tts_text]
len(tts_text), tts_text[2]

(119,
 'One of these prisons is Ukrainian mathematician, publicist, and literary scholar Leonid Plush, who believed in the possibility of building socialism with a human face. He later summed up his tragic experience of being in the walls of the terrible Dnipropetrovsk Special Psychiatric Hospital as')

In [3]:
import torch
from TTS.api import TTS
from pprint import pprint

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# List available 🐸TTS models
# pprint(TTS().list_models().list_models())

cuda


# 1. TorTuiSe

## Full Voice Over

In [4]:
# with open("full_text.txt", 'r', encoding='utf-8') as f:
#     text = ' '.join([l for l in f.readlines()])
# text = text.replace("\u200b", " ")
# text[:300]

In [5]:
torch.cuda.is_available()  

True

In [6]:
from TTS.api import TTS
tts = TTS("tts_models/en/multi-dataset/tortoise-v2").to(device)

tts.tts_to_file(text=tts_text[0],
                file_path="full_scale_output.wav",
                voice_dir="data/voices",
                speaker="many_short",
                preset="standard")

 > tts_models/en/multi-dataset/tortoise-v2 is already downloaded.
 > Using model: tortoise


KeyboardInterrupt: 

In [ ]:
from IPython.display import Audio, display

display(Audio(filename='full_scale_output.wav'))